# Downloading the Data based on Category:

In [46]:
import os
import time
import requests
import pandas as pd

def get_coin_data(category):
    url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&category={category}&order=market_cap_desc&per_page=100&page=1&sparkline=false&locale=en'
    response = requests.get(url)
    data = response.json()
    id_list = [coin['id'] for coin in data]
    return id_list

def get_coin_chart(id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    url = f'https://www.coingecko.com/price_charts/{id}/usd/365_days.json'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        stats_df = pd.DataFrame(data['stats'], columns=['timestamp', 'price'])
        volumes_df = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])
        stats_df['timestamp'] = pd.to_datetime(stats_df['timestamp'], unit='ms')
        volumes_df['timestamp'] = pd.to_datetime(volumes_df['timestamp'], unit='ms')
        
        #remove the last row as it is the current day and the data is incomplete
        stats_df = stats_df[:-1]
        volumes_df = volumes_df[:-1]
        
        return stats_df, volumes_df
    else:
        print(f"Request for {id} failed with status code {response.status_code}")
        return None, None

def save_data(category):
    id_list = get_coin_data(category)

    for id in id_list:
        stats_df, volumes_df = get_coin_chart(id)

        if stats_df is not None and volumes_df is not None:
            os.makedirs(f'{category}/stats', exist_ok=True)
            os.makedirs(f'{category}/volume', exist_ok=True)

            stats_df.to_json(f'{category}/stats/{id}.json', orient='records', date_format='iso')
            volumes_df.to_json(f'{category}/volume/{id}.json', orient='records', date_format='iso')

        # Sleep for 2 seconds to avoid hitting rate limits
        time.sleep(2)

# Usage:
save_data('artificial-intelligence')

Request for nuco-cloud failed with status code 403


# Read the prices on to a Dataframe

In [49]:
import glob
import pandas as pd
import os

def read_data(category):
    # Step 1: Get a list of all JSON files
    files = glob.glob(f'{category}/stats/*.json')

    # Initialize an empty DataFrame
    main_df = pd.DataFrame()

    # Step 2: For each file
    for file in files:
        try:
            # Read the JSON data into a DataFrame
            df = pd.read_json(file, orient='records')

            # Get the coin ID from the filename
            coin_id = os.path.splitext(os.path.basename(file))[0]

            # Rename the 'price' column to the coin ID
            df = df.rename(columns={'price': coin_id})

            # Merge the DataFrame with the main DataFrame
            if main_df.empty:
                main_df = df
            else:
                main_df = pd.merge(main_df, df, on='timestamp', how='outer')

        except Exception as e:
            print(f"Error reading file {file}: {e}")

    return main_df

# Usage:
df = read_data('artificial-intelligence')
df

,timestamp,echelon-prime,presearch,mbd-financials,lilai,cere-network,forta,numeraire,singularitynet,aimbot,...,ispolink,noisegpt,vaiot,origintrail,chirpley,kwai,bad-idea-ai,iexec-rlc,trace-network-labs,insightx
0,2023-03-02,3.786070,0.061145,0.000127,NaN,0.008224,0.150590,21.684500,0.544794,NaN,...,0.001541,NaN,0.166831,0.369530,0.001731,0.149172,NaN,1.999774,0.030577,NaN
1,2023-03-03,3.786070,0.060709,0.000068,NaN,0.008304,0.150500,20.467756,0.526491,NaN,...,0.001500,NaN,0.155614,0.343968,0.001752,0.150700,NaN,1.901262,0.029224,NaN
2,2023-03-04,3.101659,0.056827,0.000086,NaN,0.007654,0.152640,19.595320,0.457746,NaN,...,0.001377,NaN,0.135269,0.320692,0.001604,0.149175,NaN,1.818173,0.029891,NaN
3,2023-03-05,3.556111,0.054829,0.000075,NaN,0.007421,0.149665,19.055534,0.475851,NaN,...,0.001369,NaN,0.129841,0.337777,0.001709,0.139520,NaN,1.710606,0.027941,NaN
4,2023-03-06,3.440172,0.057254,0.000130,NaN,0.007300,0.149951,18.718434,0.465326,NaN,...,0.001413,NaN,0.126973,0.335029,0.001927,0.135851,NaN,1.731929,0.028525,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-02-25,NaN,0.065658,0.000170,NaN,0.008009,0.148881,19.586091,0.445283,NaN,...,0.001305,NaN,0.128620,0.372958,0.001687,0.150226,NaN,1.900972,0.027575,NaN
361,2023-02-26,NaN,0.061903,0.000070,NaN,0.007770,0.148984,19.094615,0.404964,NaN,...,0.001302,NaN,0.120267,0.359380,0.001587,0.158258,NaN,1.893438,0.026890,NaN
362,2023-02-27,NaN,0.067466,0.000120,NaN,0.007919,0.149936,19.903846,0.414806,NaN,...,0.001384,NaN,0.132421,0.394189,0.001622,0.162157,NaN,1.936023,0.028959,NaN
363,2023-02-28,NaN,0.062988,0.000130,NaN,0.008011,0.153042,19.595102,0.442910,NaN,...,0.001494,NaN,0.130229,0.363707,0.001680,0.157715,NaN,1.932966,0.029154,NaN


In [67]:
import pandas as pd
import os

def read_price(file):
    try:
        # Read the JSON data into a DataFrame
        df = pd.read_json(file, orient='records')

        # Convert 'timestamp' to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # Get the coin ID from the filename
        coin_id = os.path.splitext(os.path.basename(file))[0]

        # Rename the 'price' column to the coin ID
        df = df.rename(columns={'price': coin_id})

        return df

    except Exception as e:
        print(f"Error reading file {file}: {e}")
        return None

# Usage:
lilai_df = read_price('artificial-intelligence/stats/lilai.json')
aim_df = read_price('artificial-intelligence/stats/aimbot.json')

# Sort the dataframes by 'timestamp'
lilai_df = lilai_df.sort_values('timestamp')
aim_df = aim_df.sort_values('timestamp')

# Merge the dataframes on 'timestamp' with an outer join
merged_df = pd.merge(lilai_df, aim_df, on='timestamp', how='outer')

# Sort the merged dataframe by 'timestamp'
merged_df = merged_df.sort_values('timestamp')
merged_df

,timestamp,lilai,aimbot
0,2023-06-03,0.000761,NaN
1,2023-06-04,0.000761,NaN
2,2023-06-05,0.000755,NaN
3,2023-06-06,0.000646,NaN
4,2023-06-07,0.000690,NaN
...,...,...,...
195,2023-12-29,0.004044,9.150465
196,2023-12-30,0.004123,8.580546
197,2023-12-31,0.004060,8.730326
198,2024-01-01,0.003813,8.059692


In [68]:
get_coin_data('artificial-intelligence')

['render-token',
 'bittensor',
 'fetch-ai',
 'akash-network',
 'corgiai',
 'singularitynet',
 'autonolas',
 'echelon-prime',
 'ocean-protocol',
 'origintrail',
 'golem',
 'aioz-network',
 '0x0-ai-ai-smart-contract',
 'numeraire',
 'insure',
 'oraichain-token',
 'iexec-rlc',
 'paal-ai',
 'arkham',
 'everipedia',
 'alethea-artificial-liquid-intelligence-token',
 'humans-ai',
 'rss3',
 'dynex',
 'chaingpt',
 'arc',
 'vaiot',
 'phoenix-global',
 'forta',
 'singularitydao',
 'clore-ai',
 'selfkey',
 'tokenfi',
 'altered-state-token',
 'gameswift',
 'limewire-token',
 'optimus-ai',
 'hera-finance',
 'cere-network',
 'aleph',
 'jackal-protocol',
 'vectorspace',
 'phantasma',
 'imgnai',
 'bad-idea-ai',
 'noisegpt',
 'vaulttech',
 'octaspace',
 'presearch',
 'nuco-cloud',
 'snailbrook',
 'kizuna',
 'synesis-one',
 'mozaic',
 'thought',
 'not-financial-advice',
 'botto',
 'matrix-ai-network',
 'roko-network',
 'dtravel',
 'checkdot',
 'aimbot',
 'deepbrain-chain',
 'bostrom',
 'dexcheck',
 'pink

In [72]:
def merge_all_coins(coin_ids, base_path='artificial-intelligence/stats/'):
    # Initialize an empty DataFrame
    merged_df = pd.DataFrame()

    for coin_id in coin_ids:
        # Construct the file path
        file = os.path.join(base_path, f"{coin_id}.json")

        # Read the coin data
        coin_df = read_price(file)

        # If read_price returned None, skip this coin
        if coin_df is None:
            print(f"Skipping {coin_id} because its data could not be read")
            continue

        # If this is the first coin, assign its data to merged_df
        if merged_df.empty:
            merged_df = coin_df
        else:
            # Otherwise, merge the coin data into merged_df
            merged_df = pd.merge(merged_df, coin_df, on='timestamp', how='outer')

    # Sort the merged dataframe by 'timestamp'
    merged_df = merged_df.sort_values('timestamp')

    return merged_df

# Usage:
coin_ids = get_coin_data('artificial-intelligence')
merged_df = merge_all_coins(coin_ids)

merged_df.dropna(inplace=True, axis=1)

merged_df

Error reading file artificial-intelligence/stats/nuco-cloud.json: File artificial-intelligence/stats/nuco-cloud.json does not exist
Skipping nuco-cloud because its data could not be read


,timestamp,render-token,fetch-ai,akash-network,singularitynet,ocean-protocol,origintrail,golem,aioz-network,numeraire,...,chirpley,dotmoovs,paint,purefi,gny,cindicator,trava-finance,bird-money,trace-network-labs,all-in
0,2023-01-03,0.417047,0.096981,0.199982,0.045797,0.167662,0.162961,0.182256,0.031979,12.501519,...,0.001824,0.002886,0.000026,0.006520,0.010512,0.000384,0.000254,6.113176,0.061517,0.237539
1,2023-01-04,0.408210,0.102261,0.202916,0.045445,0.164859,0.160494,0.180084,0.032170,12.503975,...,0.001829,0.002953,0.000025,0.006676,0.010676,0.000386,0.000260,6.084369,0.053200,0.185375
2,2023-01-05,0.414987,0.112782,0.204710,0.048165,0.177884,0.169123,0.184235,0.032257,12.806308,...,0.001858,0.002971,0.000026,0.006561,0.010792,0.000401,0.000274,6.543960,0.046545,0.193798
3,2023-01-06,0.419425,0.106699,0.203696,0.046302,0.183802,0.165895,0.183518,0.031674,12.627614,...,0.001800,0.002960,0.000027,0.006416,0.010475,0.000380,0.000262,6.558734,0.044810,0.193901
4,2023-01-07,0.418794,0.125245,0.205523,0.052469,0.190703,0.166453,0.185927,0.031636,12.972412,...,0.001839,0.002971,0.000028,0.006562,0.010519,0.000316,0.000268,6.412450,0.045927,0.183230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-29,4.561565,0.686981,2.368560,0.325946,0.525559,0.599022,0.246261,0.125577,17.215752,...,0.006973,0.004069,0.000283,0.043642,0.013895,0.001072,0.001033,15.468657,0.008245,0.747143
361,2023-12-30,4.607544,0.687022,2.311316,0.324139,0.516077,0.599876,0.242054,0.138144,18.513044,...,0.006823,0.003855,0.000284,0.043544,0.012312,0.001007,0.000968,15.247491,0.008520,0.724286
362,2023-12-31,4.554695,0.689915,2.260845,0.326748,0.514239,0.672221,0.240396,0.132326,20.055389,...,0.006709,0.003869,0.000290,0.041161,0.012927,0.000868,0.000926,14.280022,0.008510,0.754416
363,2024-01-01,4.468166,0.672715,2.436958,0.318929,0.506784,0.691681,0.240318,0.141277,20.407695,...,0.006658,0.003814,0.000261,0.040696,0.012066,0.000833,0.000871,14.446346,0.009540,0.756806
